In [1]:
import os
import pandas as pd

# Path to the dataset folder (update this with your actual dataset folder path)
dataset = ['A01 - Walking', 'A02 - Sitting', 'A03 - Standing', 'A04 - Jogging', 'A05 - CrossLeg', 'A06 - Laying', 'A07 - CirWalk', 'A08 - StairUp','A09 - StairDown', 'A10 - SitUp']

dataset_path ="./FLAAP"

# Output dataframe
all_data = []

# Iterate through each activity folder
for activity_dir in os.listdir(dataset_path):
    activity_path = os.path.join(dataset_path, activity_dir)

    # Skip if it's not a directory
    if not os.path.isdir(activity_path):
        continue

    # Extract activity label from directory name (e.g., "A01 - Walking" -> "Walking")
    activity_label = activity_dir.split(" - ")[1]

    # Paths to the accelerometer and gyroscope CSV files
    accel_file = os.path.join(activity_path, "Trimmed - Accelerometer Data.csv")
    gyro_file = os.path.join(activity_path, "Trimmed - Gyroscope Data.csv")

    # Check if both files exist
    if os.path.exists(accel_file) and os.path.exists(gyro_file):
        # Read the CSV files
        accel_data = pd.read_csv(accel_file)
        gyro_data = pd.read_csv(gyro_file)

        # Merge the accelerometer and gyroscope data
        merged_data = pd.concat([accel_data, gyro_data], axis=1).drop(['Activity'], axis=1)

        # Add the label column
        merged_data["Label"] = activity_label

        # Append to the output list
        all_data.append(merged_data)

# Combine all data into a single DataFrame
final_dataset = pd.concat(all_data, ignore_index=True)

# Save the aggregated dataset to a CSV file
output_file = "aggregated_dataset.csv"
final_dataset.to_csv(output_file, index=False)

print(f"Aggregated dataset saved to {output_file}")

Aggregated dataset saved to aggregated_dataset.csv


In [2]:
final_dataset

,TimestampAcc,Acc_X,Acc_Y,Acc_Z,TimestampGyr,Gyr_X,Gyr_Y,Gyr_Z,Label
0,1.646240e+12,2.396596,7.656657,0.433351,1.646240e+12,-0.917193,-0.042078,1.821602,Walking
1,1.646240e+12,2.274492,7.716512,0.524330,1.646240e+12,-0.903877,-0.131560,1.839179,Walking
2,1.646240e+12,2.197877,7.706935,0.610521,1.646240e+12,-0.884170,-0.206661,1.847701,Walking
3,1.646240e+12,2.149993,7.647080,0.696713,1.646240e+12,-0.860201,-0.272708,1.851430,Walking
4,1.646240e+12,2.015918,7.647080,0.804452,1.646240e+12,-0.833037,-0.322775,1.849299,Walking
...,...,...,...,...,...,...,...,...,...
1061595,1.650000e+12,-1.216254,2.284068,9.471462,1.650000e+12,0.055926,1.005610,-0.011718,SitUp
1061596,1.650000e+12,-1.244985,2.324770,9.313444,1.650000e+12,0.047937,0.945955,-0.038350,SitUp
1061597,1.650000e+12,-1.240196,2.348712,9.263166,1.650000e+12,0.023968,0.885767,-0.079362,SitUp
1061598,1.650000e+12,-0.833182,2.161964,9.325416,1.650000e+12,-0.004794,0.833569,-0.120908,SitUp


In [3]:
final_dataset.Label.value_counts()

Label
Walking      205600
Jogging      158500
Sitting      144900
CirWalk      143800
CrossLeg     130300
StairDown    103300
Laying        97300
SitUp         77900
Name: count, dtype: int64

In [4]:
final_dataset.shape

(1061600, 9)

# Train

In [5]:
final_dataset.columns

Index(['TimestampAcc', 'Acc_X', 'Acc_Y', 'Acc_Z', 'TimestampGyr', 'Gyr_X',
       'Gyr_Y', 'Gyr_Z', 'Label'],
      dtype='object')

In [6]:
X = final_dataset[['Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z']]
y = final_dataset['Label'].to_numpy()

In [7]:
X

,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z
0,2.396596,7.656657,0.433351,-0.917193,-0.042078,1.821602
1,2.274492,7.716512,0.524330,-0.903877,-0.131560,1.839179
2,2.197877,7.706935,0.610521,-0.884170,-0.206661,1.847701
3,2.149993,7.647080,0.696713,-0.860201,-0.272708,1.851430
4,2.015918,7.647080,0.804452,-0.833037,-0.322775,1.849299
...,...,...,...,...,...,...
1061595,-1.216254,2.284068,9.471462,0.055926,1.005610,-0.011718
1061596,-1.244985,2.324770,9.313444,0.047937,0.945955,-0.038350
1061597,-1.240196,2.348712,9.263166,0.023968,0.885767,-0.079362
1061598,-0.833182,2.161964,9.325416,-0.004794,0.833569,-0.120908


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
from ELM import ML_FELM, ELMClassifier

In [22]:
 print("ELM")
elm = ELMClassifier(n_hidden=50, activation="relu")
elm.fit(X_train, y_train)
y_pred = elm.predict(X_test)
# print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")

print("\n============================")
print("ML-FELM")
layer_sizes = [64, 32, 16]  # Number of hidden nodes in each layer
fisher_lambdas = [0.1, 0.05, 0.01]  # Fisher regularization for each layer

# Train ML-FELM
ml_felm = ML_FELM(layer_sizes, fisher_lambdas, activation="relu")
ml_felm.fit(X_train, y_train)
y_pred = ml_felm.predict(X_test)

# Evaluate accuracy
# print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(f"Test Accuracy: {accuracy:.2f}")

ELM
              precision    recall  f1-score   support

     CirWalk       0.38      0.41      0.39     28760
    CrossLeg       0.60      0.20      0.30     26060
     Jogging       0.53      0.49      0.51     31700
      Laying       0.50      0.45      0.48     19460
       SitUp       0.54      0.16      0.25     15580
     Sitting       0.41      0.63      0.49     28980
   StairDown       0.46      0.13      0.21     20660
     Walking       0.35      0.61      0.45     41120

    accuracy                           0.42    212320
   macro avg       0.47      0.38      0.38    212320
weighted avg       0.46      0.42      0.40    212320

Accuracy: 0.42

ML-FELM
              precision    recall  f1-score   support

     CirWalk       0.40      0.41      0.40     28760
    CrossLeg       0.59      0.13      0.21     26060
     Jogging       0.52      0.59      0.55     31700
      Laying       0.50      0.36      0.42     19460
       SitUp       0.46      0.20      0.28     15